# Road Traffic Vehicle Counting Exam

In this exam, you will use the YOLO (You Only Look Once) object detection model to analyze a video of road traffic. Your primary task is to count the number of vehicles exiting the road at each of the four exits (located at the top, bottom, left, and right of the frame). The video for this task is provided at `Datasets/YOLO_Exam_Video.mp4`.

## Objectives
- Load and process the video using OpenCV.
- Utilize the YOLO model to detect vehicles in each frame.
- Manipulate the video frames using OpenCV.
- Track vehicles as they move across the video frames.
- Count the number of vehicles exiting via the top street.
- Count the number of vehicles exiting via the bottom street.
- Count the number of vehicles exiting via the left street.
- Count the number of vehicles exiting via the right street.


## Setup Environment

Before you begin, ensure you have the necessary libraries installed. You will need `opencv`, and `ultralytics` among others.
If these are not installed, you should install them.

In [35]:
!pip install ultralytics
import pandas as pd
import numpy as np
from ultralytics import YOLO
import cv2



## Load the YOLO Model

You will first need to load the YOLO model, there is a custom model for this exam called `YOLO_Model.pt` under Datasets file ⚠️ PLEASE USE THIS MODEL AND DO NOT USE ANY OTHER MODEL ⚠️.
Write the code to load the YOLO model below:

In [36]:
model = YOLO('/content/YOLO_Model.pt')


## Prepare the Video Capture

Create a variable to capture the video frames, you can use `cv2.VideoCapture()` to achive this.

In [37]:
cap = cv2.VideoCapture('/content/YOLO_Video.mp4')
cap.isOpened()





True

## Get Video Information

You can use `cv2` library to get these information fro the `VideoCapture()` variable you created to extract these information:
* `height`: Video's height.
* `width`: Video's width.
* `fps`: Video's frames.

In [38]:
h,w,fps=cap.get(cv2.CAP_PROP_FRAME_HEIGHT),cap.get(cv2.CAP_PROP_FRAME_HEIGHT),cap.get(cv2.CAP_PROP_FPS)

## Prepare Video Writer to Store the Output

Create a variable that uses `cv2.VideoCapture()` to save the video with the bounding boxes, specified Region of Interest rectangle (ROI) and the counted vehicles on each sides. You will need to make the video with the same `fps`, `width`, `height`, and specify the codec and output path of the video.

In [39]:
fourcc = cv2.VideoWriter_fourcc(*'MP4V')
out = cv2.VideoWriter('output.mp4' , fourcc,fps, (int(w),int(h)))

## Expermint to get the `(x1, y1, x2, y2)` of the ROI

### Hint
When defining the Regions of Interest (ROIs) for detecting vehicles exiting the road, consider the coordinates that outline the relevant areas. These coordinates can be specified as `(x1, y1, x2, y2)`, where `(x1, y1)` is the top-left corner and `(x2, y2)` is the bottom-right corner of the rectangle.

Experiment with different coordinates to accurately cover the streets where vehicles exit. Visualize these ROIs by drawing rectangles on a frame of the video and adjust the coordinates as needed until you capture the desired areas effectively.

In [40]:
x1_top,y1_top,x2_top,y2_top = 0,0,int(w),int(h/2)-100
x1_bottom,y1_bottom , x2_bottom ,y2_bottom = 0, int(h/2)+100 , int(w),int(h)
x1_left,y1_left,x2_left,y2_left = 0,0, int(w/2)-100,int(h)
x1_right,y1_right,x2_right,y2_right = int(w/2)+100,0,int(w) , int(h)

## Process Video Frames and Identify Vehicles Left the Road Form Each Street

For each frame in the video, use the YOLO model to detect and track vehicles. You will need to write a loop that processes each frame and applies the YOLO model.

In each frame, after detecting the vehicles, determine whether they exited the road via the top, bottom, left, or right street. You can use the positions of the bounding boxes provided by YOLO to do this.

- The video should display bounding boxes around the detected objects.
- The video should display your name on the top.
- It should display the calculated center of each vehicle.
- The video should display the confidence score, alongside the object ID and class ID of each detected and tracked object.
- The video should display the rectangles representing the regions where you count the vehicles that have crossed and exited.
- The video should display the number of vehicles that exited via the top street.
- The video should display the number of vehicles that exited via the bottom street.
- The video should display the number of vehicles that exited via the left street.
- The video should display the number of vehicles that exited via the right street.

In [ ]:
from collections import defaultdict

top_count = 0
bottom_count = 0
left_count = 0
right_count = 0

exited_vehicle_ids = set()

vehicle_positions = defaultdict(list)

while True:
  ret, frame = cap.read()
  if not ret:
    break

  results = model(frame)
  for result in results:
    boxes = result.boxes.cpu().numpy()
    for box in boxes:
      x1, y1, x2, y2 = box.xyxy[0].astype(int)
      cx = int((x1 + x2) / 2)
      cy = int((y1 + y2) / 2)
      conf = round(box.conf[0], 2)
      cls = int(box.cls[0])

      if box.id is not None:
        id = int(box.id[0])
      else:
        id = 0

      cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
      cv2.circle(frame, (cx, cy), 5, (0, 0, 255), -1)

      label = f"ID: {id} Conf: {conf} Class: {cls}"
      cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

      vehicle_positions[id].append((cx, cy))

      if id not in exited_vehicle_ids:
        if y2 < y1_top and cx > x1_top and cx < x2_top:
          top_count += 1
          exited_vehicle_ids.add(id)

          cv2.rectangle(frame, (x1_top, y1_top), (x2_top, y2_top), (255, 0, 0), 2)
          cv2.rectangle(frame, (x1_bottom, y1_bottom), (x2_bottom, y2_bottom), (255, 0, 0), 2)
          cv2.rectangle(frame, (x1_left, y1_left), (x2_left, y2_left), (255, 0, 0), 2)
          cv2.rectangle(frame, (x1_right, y1_right), (x2_right, y2_right), (255, 0, 0), 2)

          cv.putText(frame, f"Top: {top_count}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)
          cv.putText(frame, f"Bottom: {bottom_count}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)
          cv.putText(frame, f"Left: {left_count}", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)
          cv.putText(frame, f"Right: {right_count}", (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)

          cv.putText(frame, "detect", (int(w) - 200, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
          out.write






0: 384x640 2 buss, 90 cars, 2 trucks, 2 vans, 2992.4ms
Speed: 5.4ms preprocess, 2992.4ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 92 cars, 2 trucks, 3 vans, 3580.3ms
Speed: 3.9ms preprocess, 3580.3ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 94 cars, 2 trucks, 2 vans, 2692.1ms
Speed: 3.7ms preprocess, 2692.1ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 92 cars, 2 trucks, 2 vans, 2689.5ms
Speed: 4.2ms preprocess, 2689.5ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 94 cars, 2 trucks, 2 vans, 2638.5ms
Speed: 4.3ms preprocess, 2638.5ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 92 cars, 2 trucks, 2 vans, 3930.9ms
Speed: 3.8ms preprocess, 3930.9ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 92 cars, 2 trucks, 2 vans, 2686.8ms
Speed: 4.5ms p

In [ ]:
out.release()


## Save and Submit Your Work

Submit both the notebook and the output video